In [3]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import re
import collections
import statistics

In [11]:
df_doc = pd.read_excel("https://github.com/tbwa-digital-strategy/Blue-Bird-Bio/blob/master/HCPStateLevelData.xlsx?raw=true")
df_med_cent = pd.read_excel("https://github.com/tbwa-digital-strategy/Blue-Bird-Bio/blob/master/medical%20centers.xlsx?raw=true")
df_pop = pd.read_csv("https://raw.githubusercontent.com/tbwa-digital-strategy/Blue-Bird-Bio/master/scprc-est2016-18%2Bpop-res.csv")

# print(list(df_med_cent['data'].values))
# use only one of the following lines, whichever you prefer
# for i in list(df_med_cent['data'].values):
#     print(re.findall("[(][\d]{3}[)][ ]?[\d]{3}-[\d]{4}", str(i)))
filtered = [i for i in list(df_med_cent['data'].values) if len(re.findall(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", str(i))) == 0]

revised_filtered_lst = []

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

df_doc["code"] = df_doc["State"].apply(lambda x: x.split(",")[1])

for code in df_doc["code"]:
    if code not in bonemarrow_nums_by_state:
        bonemarrow_nums_by_state[code] = 0
        
print(df_doc.columns)

curr_nat_avg = 20.16

avg = statistics.mean(df_doc["BR (35-50)"])
std = statistics.stdev(df_doc["BR (35-50)"])

br_mtp = []

add_const = ((24.3/26)*(8)+(.5/26)*8+6.4/26*8)/129

for num in df_doc["BR (35-50)"]:
    br_mtp.append((avg-num)/std)
    

curr_elder_women_br_estimate = []

for mtp in br_mtp:
    if mtp > 0:
        curr_elder_women_br_estimate.append(curr_nat_avg + mtp*curr_nat_avg/avg*std)
    elif mtp < 0:
        curr_elder_women_br_estimate.append(curr_nat_avg + mtp*std + add_const)
    
df_doc["BR (35-50) (current)"] = curr_elder_women_br_estimate

print(curr_elder_women_br_estimate)

Index(['State', 'Endos', 'Opthamologists', 'Rheumatolosits ',
       'Allergy & Immunology', 'Neurologists', 'Gastroenterologists ',
       'Infectious Disease', 'Population', 'Density Endos (per capita)',
       'Density Opthamologists (per capita)', 'code',
       'Rheumatoid Arthritis (%)', 'Diabetes Mellitus (%)', 'Square Mileage',
       'Adult Smoking Rate (%)', 'Hematology Oncologists',
       'Thalassemia Patients', 'Density Thalassemia (per capita)',
       'Density Hematology Oncologists (per capita)', 'Population Density',
       'Bone Marrow Medical Centers by State',
       'Density Hematology Oncologists (per Thalassemia capita)',
       'Bone Marrow Medical Centers by Thalassemia Patients', 'BR (35-50)'],
      dtype='object')
[26.150133968883694, 23.075683981466405, 20.85355676283807, 28.05264014921617, 15.250104879160967, 17.30010487916097, 12.170104879160968, 19.450104879160968, 19.48010487916097, 18.58010487916097, 20.07010487916097, 16.30010487916097, 24.73466937071